In [3]:
import sqlite3

# Create a connection to the database
conn = sqlite3.connect('yw.db')
cursor = conn.cursor()

# Create the Books table
cursor.execute('''CREATE TABLE IF NOT EXISTS Books
                  (BookID INTEGER PRIMARY KEY,
                   Title TEXT,
                   Author TEXT,
                   ISBN TEXT,
                   Status TEXT)''')

# Create the Users table
cursor.execute('''CREATE TABLE IF NOT EXISTS Users
                  (UserID INTEGER PRIMARY KEY,
                   Name TEXT,
                   Email TEXT)''')

# Create the Reservations table
cursor.execute('''CREATE TABLE IF NOT EXISTS Reservations
                  (ReservationID INTEGER PRIMARY KEY,
                   BookID INTEGER,
                   UserID INTEGER,
                   ReservationDate TEXT)''')

# Function to add a new book to the database
def add_book(title, author, isbn, status):
    cursor.execute("INSERT INTO Books (Title, Author, ISBN, Status) VALUES (?, ?, ?, ?)",
                   (title, author, isbn, status))
    conn.commit()
    print("Book added successfully.")

# Function to find a book's details based on BookID
def find_book_details(book_id):
    cursor.execute('''SELECT Books.Title, Books.Status, Users.Name
                      FROM Books
                      LEFT JOIN Reservations ON Books.BookID = Reservations.BookID
                      LEFT JOIN Users ON Reservations.UserID = Users.UserID
                      WHERE Books.BookID = ?''', (book_id,))
    result = cursor.fetchone()
    if result:
        title, status, user_name = result
        print(f"Title: {title}")
        print(f"Status: {status}")
        if user_name:
            print(f"Reserved by: {user_name}")
    else:
        print("Book not found.")

# Function to find a book's reservation status based on BookID, Title, UserID, and ReservationID
def find_reservation_status(text):
    if text.startswith('LB'):
        cursor.execute('''SELECT Books.Status, Users.Name
                          FROM Books
                          LEFT JOIN Reservations ON Books.BookID = Reservations.BookID
                          LEFT JOIN Users ON Reservations.UserID = Users.UserID
                          WHERE Books.BookID = ?''', (text,))
    elif text.startswith('LU'):
        cursor.execute('''SELECT Books.Status, Books.Title
                          FROM Books
                          LEFT JOIN Reservations ON Books.BookID = Reservations.BookID
                          LEFT JOIN Users ON Reservations.UserID = Users.UserID
                          WHERE Users.UserID = ?''', (text,))
    elif text.startswith('LR'):
        cursor.execute('''SELECT Books.Status, Books.Title, Users.Name
                          FROM Books
                          LEFT JOIN Reservations ON Books.BookID = Reservations.BookID
                          LEFT JOIN Users ON Reservations.UserID = Users.UserID
                          WHERE Reservations.ReservationID = ?''', (text,))
    else:
        cursor.execute('''SELECT Books.Status, Books.Title
                          FROM Books
                          WHERE Books.Title = ?''', (text,))

    result = cursor.fetchone()
    if result:
        status, title = result
        print(f"Title: {title}")
        print(f"Status: {status}")
    else:
        print("Book not found.")

# Function to find all the books in the database
def find_all_books():
    cursor.execute('''SELECT Books.Title, Books.Author, Books.ISBN, Books.Status, Users.Name
                      FROM Books
                      LEFT JOIN Reservations ON Books.BookID = Reservations.BookID
                      LEFT JOIN Users ON Reservations.UserID = Users.UserID''')
    results = cursor.fetchall()
    if results:
        for result in results:
            title, author, isbn, status, user_name = result
            print(f"Title: {title}")
            print(f"Author: {author}")
            print(f"ISBN: {isbn}")
            print(f"Status: {status}")
            if user_name:
                print(f"Reserved by: {user_name}")
            print("------------")
    else:
        print("No books found.")

# Function to modify/update book details based on its BookID
def update_book_details(book_id, new_status):
    cursor.execute("UPDATE Books SET Status = ? WHERE BookID = ?", (new_status, book_id))
    cursor.execute("UPDATE Reservations SET Status = ? WHERE BookID = ?", (new_status, book_id))
    conn.commit()
    print("Book details updated successfully.")

# Function to delete a book based on its BookID
def delete_book(book_id):
    cursor.execute("DELETE FROM Books WHERE BookID = ?", (book_id,))
    cursor.execute("DELETE FROM Reservations WHERE BookID = ?", (book_id,))
    conn.commit()
    print("Book deleted successfully.")

# Main program loop
while True:
    print("Library Management System")
    print("-------------------------")
    print("1. Add a new book")
    print("2. Find a book's details based on BookID")
    print("3. Find a book's reservation status")
    print("4. Find all books in the database")
    print("5. Update book details")
    print("6. Delete a book")
    print("7. Exit")

    choice = input("Enter your choice (1-7): ")

    if choice == '1':
        title = input("Enter the book's title: ")
        author = input("Enter the author's name: ")
        isbn = input("Enter the book's ISBN: ")
        status = input("Enter the book's status: ")
        add_book(title, author, isbn, status)
    elif choice == '2':
        book_id = input("Enter the BookID: ")
        find_book_details(book_id)
    elif choice == '3':
        text = input("Enter the BookID, Title, UserID, or ReservationID: ")
        find_reservation_status(text)
    elif choice == '4':
        find_all_books()
    elif choice == '5':
        book_id = input("Enter the BookID: ")
        new_status = input("Enter the new status: ")
        update_book_details(book_id, new_status)
    elif choice == '6':
        book_id = input("Enter the BookID: ")
        delete_book(book_id)
    elif choice == '7':
        break
    else:
        print("Invalid choice. Please try again.")

# Close the database connection
conn.close()

Library Management System
-------------------------
1. Add a new book
2. Find a book's details based on BookID
3. Find a book's reservation status
4. Find all books in the database
5. Update book details
6. Delete a book
7. Exit
Book deleted successfully.
Library Management System
-------------------------
1. Add a new book
2. Find a book's details based on BookID
3. Find a book's reservation status
4. Find all books in the database
5. Update book details
6. Delete a book
7. Exit


In [1]:
from nltk.corpus import gutenberg
from gensim import corpora
import gensim
import pyLDAvis.gensim
alice = gutenberg.sents('carroll-alice.txt')
dictionary = corpora.Dictionary(alice)
corpus = [dictionary.doc2bow(text) for text in alice]
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = 3, id2word = dictionary, passes = 20)
print(ldamodel.print_topics(num_topics = 3, num_words = 3))
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary)
vis

[(0, '0.053*"," + 0.036*"the" + 0.033*"."'), (1, '0.114*"\'" + 0.038*"I" + 0.037*","'), (2, '0.067*"," + 0.053*"the" + 0.036*"and"')]


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2     -0.107061 -0.071117       1        1  50.599519
1      0.144324 -0.027335       2        1  37.912317
0     -0.037264  0.098452       3        1  11.488164, topic_info=     Term         Freq        Total Category  logprob  loglift
0       '  1586.000000  1586.000000  Default  30.0000  30.0000
14      I   500.000000   500.000000  Default  29.0000  29.0000
20      ,  1838.000000  1838.000000  Default  28.0000  28.0000
12      .   697.000000   697.000000  Default  27.0000  27.0000
306   you   308.000000   308.000000  Default  26.0000  26.0000
..    ...          ...          ...      ...      ...      ...
176  went    18.407674    76.269166   Topic3  -5.3608   0.7424
132   all    21.602354   159.831456   Topic3  -5.2008   0.1625
65     as    20.436920   229.231706   Topic3  -5.2562  -0.2535
21     ,'    17.365926   363.868122   Topic3  -5.4191  -0.8784
57   very    15.791993   116.616211   Topic3  -5.5141   0.1645

[232 rows x 6 columns], token_table=      Topic      Freq  Term
term                       
98        1  0.312902     !
98        2  0.646664     !
98        3  0.048674     !
116       1  0.011702    !'
116       2  0.982985    !'
...     ...       ...   ...
306       2  0.717001   you
306       3  0.259548   you
571       1  0.020597  your
571       2  0.803302  your
571       3  0.185377  your

[354 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 2, 1])